In [1]:
import os
import timm
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as T
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:

file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_trait_weather.csv"

df = pd.read_csv(file_path)

df = df.sample(n=1000, random_state=42)

df.head(3)


,ImageFilePath,Genus,ScientificName,Length,Width,TempMin,TempMax,Precipitation,Rain,Snowfall,WindGust,WindSpeed,TempAvg
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Synuchus impunctatus,0.530811,0.242519,27.3,18.9,18.7,18.7,0.0,38.5,11.6,23.10
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Agonum punctiforme,0.457965,0.243925,29.7,16.6,0.0,0.0,0.0,36.4,18.1,23.15
304,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Pterostichus,Pterostichus melanarius,0.994307,0.569771,23.2,15.6,7.2,7.2,0.0,47.9,15.2,19.40


In [3]:

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("hf-hub:imageomics/bioclip")

tokenizer = open_clip.get_tokenizer("hf-hub:imageomics/bioclip")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

for param in model.parameters() :
    param.requires_grad = False
    

In [4]:

def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    image = preprocess_val(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    
    return embedding.cpu().numpy().flatten()


In [5]:
df.columns

Index(['ImageFilePath', 'Genus', 'ScientificName', 'Length', 'Width',
       'TempMin', 'TempMax', 'Precipitation', 'Rain', 'Snowfall', 'WindGust',
       'WindSpeed', 'TempAvg'],
      dtype='object')

In [6]:
df['Embedding'] = [extract_features(img) for img in tqdm(df['ImageFilePath'], desc='Extracting Embeddings')]

X = np.vstack(df['Embedding'].values)
numeric_cols = df.loc[:, 'ScientificName':].select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'ScientificName']
X = np.hstack((X, df[numeric_cols].values))

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(df['ScientificName'])


Extracting Embeddings: 100%|██████████| 1000/1000 [02:15<00:00,  7.39it/s]


In [7]:
df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 77.50% | Prec: 66.03% | Rec: 77.50% | F1: 70.45% | Bal Acc: 47.75% | MCC: 0.7592


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


LogisticRegression        | Acc: 84.50% | Prec: 79.58% | Rec: 84.50% | F1: 80.64% | Bal Acc: 65.24% | MCC: 0.8347


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 76.50% | Prec: 67.17% | Rec: 76.50% | F1: 70.39% | Bal Acc: 49.98% | MCC: 0.7487


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForest              | Acc: 76.50% | Prec: 63.92% | Rec: 76.50% | F1: 68.92% | Bal Acc: 45.28% | MCC: 0.7483
MLP_Baseline              | Acc: 83.50% | Prec: 76.45% | Rec: 83.50% | F1: 79.02% | Bal Acc: 64.97% | MCC: 0.8236


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": label_encoder.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width,TempMin,TempMax,Precipitation,Rain,Snowfall,WindGust,WindSpeed,TempAvg,Embedding,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
6853,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Euryderus,Euryderus grossus,0.920304,0.643351,34.1,15.3,0.0,0.0,0.0,40.7,24.8,24.7,"[-0.13278013, 0.11599904, 0.22043616, 0.097139...",Euryderus grossus,Euryderus grossus,Euryderus grossus,Harpalus pensylvanicus,Euryderus grossus
10193,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Amara,Amara quenseli,0.477025,0.276834,33.8,20.6,0.0,0.0,0.0,33.1,14.0,27.2,"[-0.18027516, 0.32456037, -0.15323278, -0.1785...",Amara quenseli,Amara quenseli,Amara quenseli,Amara quenseli,Amara quenseli


In [10]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/BioCLIP-traits-weather.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/BioCLIP-traits-weather-metrics.csv", index=False)